# 2.16 Pandas. Продолжаем работу с датами

## Датасет содержит данные о продажах в булочной.
В каждой строчке содержится информация об отдельном купленном товаре.
TransactionNo - номер покупки (может повторяться так как могли купить сразу несколько товаров)
Items - товар, который был куплен
DateTime - время покупки

In [1]:
import pandas as pd

2.16.1 По файлу определите кол-во видов товаров в булочной

In [2]:
df_bakery = pd.read_csv('./content/bakery.txt', parse_dates=['DateTime'])

In [4]:
df_bakery.Items.nunique()

94

2.16.2 По файлу определите сколько чая (Tea) было продано в булочной за все время

In [8]:
df_bakery.query("Items == 'Tea'").Items.count()

1435

2.16.3 По файлу определите сколько покупок было совершено первого мая 2017 года.
Кол-во покупок != кол-во строк.
Кол-во покупок == Кол-во уникальных значений TransactionNo

In [20]:
ts = pd.Timestamp(2017, 5, 1)
df_bakery['DateTime'] = pd.to_datetime(df_bakery.DateTime)

In [33]:
df_bakery[df_bakery.DateTime.dt.date == ts.date()].TransactionNo.nunique()

50

2.16.4 По файлу определите, сколько покупок было совершено в понедельники. Для этого:
Преобразуйте колонку в тип datetime;
Создайте новую колонку, в которой будет информация о том, являлся ли день покупки понедельником;
Посчитайте кол-во уникальных значений в TransactionNo - это и будет кол-во покупок.

In [41]:
df_bakery['is_monday'] = df_bakery.DateTime.dt.day_name() == 'Monday'
df_bakery.query('is_monday')['TransactionNo'].nunique()

1441

2.16.5 По файлу определите, сколько покупок было совершено в выходные. Для этого:
Преобразуйте колонку в тип datetime;
Создайте новую колонку, в которой будет информация о том, являлся ли день покупки выходным;
Посчитайте кол-во уникальных значений в TransactionNo - это и будет кол-во покупок.

In [47]:
df_bakery['is_weekend'] = df_bakery.DateTime.dt.dayofweek.isin([5, 6])
df_bakery.query('is_weekend')['TransactionNo'].nunique()

2962

2.16.6 По файлу определите, в какой месяц какого года совершилось наибольшее кол-во покупок (TransactionNo).
Преобразуйте колонку в тип datetime и сделайте ее индексом датафрейма;
Сделайте resample с периодичностью в 1 месяц;
Посчитайте кол-во уникальных значений в TransactionNo - это и будет кол-во покупок в каждом месяце;
Найдите в результате месяц где покупок больше всего.
Ответ запишите в формате YYYY-MM.

In [8]:
df_bakery.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20507 entries, 0 to 20506
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   TransactionNo  20507 non-null  int64         
 1   Items          20507 non-null  object        
 2   DateTime       20507 non-null  datetime64[ns]
dtypes: datetime64[ns](1), int64(1), object(1)
memory usage: 480.8+ KB


In [21]:
t = df_bakery.set_index('DateTime').resample('1M').count()
print(t['TransactionNo'].idxmax().strftime('%Y-%m'))

2017-03


## Теперь более простой датасет. Содержит данные о поездках в такси в очень упрощенном виде.

Для решения задач нужно будет произвести небольшую обработку.
Сложить столбцы DATE и TIME просто через +. Чтобы получить полную строку с датой
Новую колонку перевести в тип datetime через pd.to_datetime
Результат записать в индекс датафрейма

In [29]:
df_uber = pd.read_csv('./content/uber.txt')
df_uber.head()

,DATE,TIME,PICK UP ADDRESS
0,7/1/2014,12:00:00 AM,"874 E 139th St Mott Haven, BX"
1,7/1/2014,12:01:00 AM,"628 E 141st St Mott Haven, BX"
2,7/1/2014,12:01:00 AM,"601 E 156th St South Bronx, BX"
3,7/1/2014,12:01:00 AM,"708 E 138th St Mott Haven, BX"
4,7/1/2014,12:02:00 AM,"700 E 140th St Mott Haven, BX"


In [106]:
df_uber = df_uber.set_index(pd.to_datetime(df_uber.DATE + ' ' + df_uber.TIME))

In [215]:
df_uber.head()

,DATE,TIME,PICK UP ADDRESS
2014-07-01 00:00:00,7/1/2014,12:00:00 AM,"874 E 139th St Mott Haven, BX"
2014-07-01 00:01:00,7/1/2014,12:01:00 AM,"628 E 141st St Mott Haven, BX"
2014-07-01 00:01:00,7/1/2014,12:01:00 AM,"601 E 156th St South Bronx, BX"
2014-07-01 00:01:00,7/1/2014,12:01:00 AM,"708 E 138th St Mott Haven, BX"
2014-07-01 00:02:00,7/1/2014,12:02:00 AM,"700 E 140th St Mott Haven, BX"


2.16.7 По файлу определите на какой адрес такси вызывали наибольшее кол-во раз.

In [107]:
print(df_uber['PICK UP ADDRESS'].value_counts().idxmax())

 400 Brook Ave Mott Haven, BX


2.16.8 По файлу определите самый загруженный день.

In [108]:
print(df_uber.resample('D')['PICK UP ADDRESS'].count().idxmax().date())

2014-09-13


2.16.9 По файлу определите самую загруженную неделю. В качестве ответа укажите дату последнего дня недели (resample выдает ее по умолчанию).

In [109]:
print(df_uber.resample('W')['PICK UP ADDRESS'].count().idxmax().date())

2014-09-14


## И последний файл.

In [204]:
df_disruptions = pd.read_csv('./content/disruptions.txt')
df_disruptions.head()

,Event Description,Year,Date Event Began,Time Event Began,Date of Restoration,Time of Restoration,Respondent,Geographic Areas,NERC Region,Demand Loss (MW),Number of Customers Affected,Tags
0,Severe Weather - Thunderstorms,2014,6/30/2014,8:00 PM,7/2/2014,6:30 PM,Exelon Corporation/ComEd,Illinois,RFC,Unknown,"420,000","severe weather, thunderstorm"
1,Severe Weather - Thunderstorms,2014,6/30/2014,11:20 PM,7/1/2014,5:00 PM,Northern Indiana Public Service Company,North Central Indiana,RFC,Unknown,"127,000","severe weather, thunderstorm"
2,Severe Weather - Thunderstorms,2014,6/30/2014,5:55 PM,7/1/2014,2:53 AM,We Energies,Southeast Wisconsin,MRO,424,"120,000","severe weather, thunderstorm"
3,Physical Attack - Vandalism,2014,6/24/2014,2:54 PM,6/24/2014,2:55 PM,Tennessee Valley Authority,"Nashville, Tennessee",SERC,Unknown,Unknown,"vandalism, physical"
4,Physical Attack - Vandalism,2014,6/19/2014,8:47 AM,6/19/2014,8:48 AM,Tennessee Valley Authority,"Nashville, Tennessee",SERC,Unknown,Unknown,"vandalism, physical"


В файле содержится информация об аварийных отключениях электричества.
Колонки:
Event Description - описание происшествия
Date Event Began,Time Event Began - дата и время начала происшествия (необходимо перевести в datetime аналогично такси)
Date of Restoration,Time of Restoration - дата и время конца происшествия (необходимо перевести в datetime аналогично такси)
Respondent - ответственный
Demand Loss (MW) - потери спроса в мега ватах
Number of Customers Affected - кол-во пострадавших, т.е. у какого кол-ва людей отключилось электричество

In [206]:
df_disruptions['Began_Date_Time'] = pd.to_datetime(
    df_disruptions['Date Event Began'] + ' ' + df_disruptions['Time Event Began'])
df_disruptions['Restoration_Date_Time'] = pd.to_datetime(
    df_disruptions['Date of Restoration'] + ' ' + df_disruptions['Time of Restoration'])
df_disruptions.drop(columns=['Date Event Began', 'Time Event Began', 'Date of Restoration', 'Time of Restoration'],
                    inplace=True)

2.16.10 Напишите в какой месяц и в какой год произошло больше всего аварий

In [127]:
df_disruptions.groupby([df_disruptions.Began_Date_Time.dt.year, df_disruptions.Began_Date_Time.dt.month])[
    'Event Description'].count().idxmax()


(2012, 10)

2.16.11 В какой месяц и год среднее время устранения аварии было минимальным. Ответ запишите в таком формате: "2015-12"

In [207]:
df_disruptions['Delta_of_Restoration'] = df_disruptions.Restoration_Date_Time - df_disruptions.Began_Date_Time

In [214]:
df_disruptions[df_disruptions['Delta_of_Restoration'] > pd.Timedelta(days=0)].set_index('Began_Date_Time').resample(
    'M')['Delta_of_Restoration'].mean().idxmin().strftime('%Y-%m')

'2005-04'